In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import UnexpectedAlertPresentException
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm

In [32]:
#토지대장 열람신청 입구
url = 'https://www.gov.kr/mw/AA020InfoCappView.do?HighCtgCD=A09005&CappBizCD=13100000026&tp_seq=01'
# address = address_list[3]

In [33]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
options = Options()
options.add_argument('--start-maximized') #big size
options.add_argument(f'user-agent={headers}')
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


### 반자동화

In [34]:
#로그인 이후 돌아가기
driver.get(url)
time.sleep(0.5)
#신청하기버튼
ele = driver.find_element_by_css_selector('#applyBtn > a')
ele.click()

In [27]:
data = pd.read_excel('./토지대장/경기도 인창시 토지분석.xlsx')

In [28]:
def split_bar(x,i):
    if i == 0:
        try:
            a = x.split('-')[i]
        except:
            a = x
    else:
        try:
            a = x.split('-')[i]
        except:
            a = ''
    return a
    

In [29]:
#a_5_0 : 본번, a5_1 : 부번, a2 주소 
import re
pattern = re.compile('[0-9\-]+')

# #안나눠져 있는경우
# data['A5'] = data['A6'].apply(lambda x : re.findall(pattern, x)[0])
# a5_0 = data['본번'].apply(lambda x : split_bar(x,0))
# a5_1 = data['부번'].apply(lambda x : split_bar(x,1))
a5_0 = data['본번']
a5_1 = data['부번']
mountain = data['지목'].apply(lambda x: x[0] == '산')
a2 = data['주소']

In [35]:
error_ls = [] #에러저장
#for문 
for i in tqdm(range(0, len(data))):
    try :
        #로그인 이후 돌아가기
        driver.get(url)
        time.sleep(0.5)
        #신청하기버튼
        ele = driver.find_element_by_css_selector('#applyBtn > a')
        ele.click()
        wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')
        
        #열람의 경우(출력으로하는 경우 주석처리)
        ele = driver.find_element_by_css_selector('body > div.contentsWrap > div > div.form-inner > div.element-tab > a:nth-child(3)')
        ele.click()
        wait = WebDriverWait(driver, 360).until(EC.element_to_be_clickable((By.CLASS_NAME, 'data-row-span-2')), 'Timed out waiting for loading')

        #임야구분
        if mountain[i]:
            ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_대장구분_\.라디오코드_2')
            ele.click()

        #연혁인쇄
        ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_연혁인쇄선택_\.라디오코드_1')
        ele.click()
        #소유자 주민번호 뒷자리
        ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_소유자주번뒷자리표시\.라디오코드_1')
        ele.click()

        #검색버튼
        ele = driver.find_element_by_css_selector('#btnAddress')
        ele.click()

        #팝업창이동
        all_windows = driver.window_handles 
        parent_window = all_windows[0] 
        child_window = all_windows[1] 
        driver.switch_to.window(child_window) 
        time.sleep(1)

        #검색입력
        ele = driver.find_element_by_css_selector('#txtAddr')
        address = a2[i]
        ele.send_keys(address)
        time.sleep(1)

        #검색클릭
        ele = driver.find_element_by_css_selector('#frm_popup > fieldset > div > div > span > button')
        ele.click()
        time.sleep(1)

        #검색결과에서 유사한 주소 색출(파주시, xx리)
        a = address.split(' ')[1]
        b = b = address.split(' ')[-1]

        ele = driver.find_element_by_css_selector(f'#resultList')
        temp = ele.text.split('\n')
        temp = pd.Series([(a in x)&(b in x) for x in temp])
        postion  = temp[temp == True].index[0]+1

        ele = driver.find_element_by_css_selector(f'#resultList > a:nth-child({postion}) > dl > dd > div')
        ele.click()
        time.sleep(0.5)
        #복귀
        driver.switch_to.window(parent_window) 

        #번지호 입력
        ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_신청토지소재지_주소정보_상세주소_번지')
        ele.send_keys(str(a5_0[i]))
        ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_신청토지소재지_주소정보_상세주소_호')
        ele.send_keys(str(a5_1[i]))

        #출력신청
        ele = driver.find_element_by_css_selector('#btn_end')
        ele.click()

        #출력대기
        time.sleep(1)
        wait = WebDriverWait(driver, 180).until(EC.element_to_be_clickable((By.ID, 'srch')), 'Timed out waiting for loading')
    except UnexpectedAlertPresentException:
        error_ls.append(i)
        print(f'출력실패_{i}')
        pass

### 에러체크
* address 지정안되있었음

In [36]:
error_ls

[]

In [37]:
for i in error_ls:
    print(a5_0[i],a5_1[i],mountain[i],a2[i])

### 작업용

In [204]:
i = 60

In [205]:
address = a2[i]

In [206]:
#임야구분
if mountain[i]:
    ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_대장구분_\.라디오코드_2')
    ele.click()

#연혁인쇄
ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_연혁인쇄선택_\.라디오코드_1')
ele.click()
#소유자 주민번호 뒷자리
ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_소유자주번뒷자리표시\.라디오코드_1')
ele.click()

In [207]:
#검색버튼
ele = driver.find_element_by_css_selector('#btnAddress')
ele.click()

In [208]:
#팝업창이동
all_windows = driver.window_handles 
parent_window = all_windows[0] 
child_window = all_windows[1] 
driver.switch_to.window(child_window) 
time.sleep(1)

#검색입력
ele = driver.find_element_by_css_selector('#txtAddr')
ele.send_keys(address)
time.sleep(1)


In [209]:
#검색클릭
ele = driver.find_element_by_css_selector('#frm_popup > fieldset > div > div > span > button')
ele.click()
time.sleep(1)

In [210]:
#검색결과에서 유사한 주소 색출(파주시, xx동)
a = address.split(' ')[1]
b = address.split(' ')[2]

ele = driver.find_element_by_css_selector(f'#resultList')
temp = ele.text.split('\n')
temp = pd.Series([(a in x)&(b in x) for x in temp])
postion  = temp[temp == True].index[0]+1

ele = driver.find_element_by_css_selector(f'#resultList > a:nth-child({postion}) > dl > dd > div')
ele.click()

In [211]:
#복귀
driver.switch_to.window(parent_window) 

#번지호 입력
ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_신청토지소재지_주소정보_상세주소_번지')
bunji = a5_0[i]
print(i,bunji)
time.sleep(1)
ele.send_keys(bunji)
ele = driver.find_element_by_css_selector('#토지임야대장신청서_IN-토지임야대장신청서_신청토지소재지_주소정보_상세주소_호')
ho = a5_1[i]
ele.send_keys(ho)



60 1477


In [212]:
bunji,ho

('1477', '')

In [213]:
#출력신청
ele = driver.find_element_by_css_selector('#btn_end')
ele.click()